In [2]:
import psutil
psutil.cpu_percent()

0.0

In [5]:
import psutil
import time
import os
from IPython.display import clear_output

current_pid = os.getpid()
print(f'setting high prio for set_caiman_nice self pid = {current_pid}')
psutil.Process(os.getpid()).nice(psutil.ABOVE_NORMAL_PRIORITY_CLASS)

while True: # forever set caiman nice
# while psutil.cpu_percent() > 50: # when cpu too high, set caiman nice
    cpu_util = psutil.cpu_percent()
    print(f'cpu util = {cpu_util}, start setting caiman nice')

    process_name = "python"
    pids = []
    for proc in psutil.process_iter():
        if process_name in proc.name():
            pids.append(proc.pid)
    pids.remove(os.getpid()) # get pid for all python processes except for self

    print(f'setting low prio for current user\'s caiman process.')
    for pid in pids:
        try:
            username = psutil.Process(pid).username()
            print("lower prio of process {} owned by {}".format(pid, username))
            psutil.Process(pid).nice(psutil.BELOW_NORMAL_PRIORITY_CLASS)
        except psutil.AccessDenied: # if access denied, this pid is not owned by current user
            pass

    time.sleep(600) # check for new python process every n sec
    clear_output(wait=True) # clear cell output after a pause, otherwise too long


cpu util = 0.3, start setting caiman nice
setting low prio for all caiman process.
pid = 11416
pid = 12252
pid = 13236
pid = 13344
pid = 15452
pid = 18920
pid = 19448
pid = 19536
pid = 20392
pid = 21636
pid = 21880
pid = 23880
pid = 27392
pid = 28120
pid = 28520
pid = 29012
pid = 29916
pid = 32980
pid = 33488
pid = 33940
pid = 33948
pid = 33988
pid = 34000
pid = 34240
pid = 34516
pid = 34624
pid = 34748
pid = 34860
pid = 35588
pid = 36064
pid = 36140
pid = 36176
pid = 36380
pid = 37048
pid = 37460
pid = 38444
pid = 38712
pid = 38900
pid = 38924
pid = 39316
pid = 39648
pid = 39864
pid = 39908
pid = 39996
pid = 41556
pid = 41876
pid = 43236
pid = 43432
pid = 43512
pid = 43544
pid = 43804
pid = 44152
pid = 44432
pid = 44856
pid = 44976
pid = 45008


AccessDenied: psutil.AccessDenied (pid=45008)